In [12]:
import sys
import os
import numpy as np
from trodes import read_exported as tre
sys.path.append(r'c:\Users\zhaoz\Desktop\Research\SocialMemory\diff_fam_social_memory_ephys')

In [13]:
box_to_ecu_dict = {
            1: {'dio_ECU_Din1': 'selfish light',
                'dio_ECU_Din2': 'coop light',
                'dio_ECU_Din6': 'selfish nose poke',
                'dio_ECU_Din10': 'coop nose poke',
                'dio_ECU_Din8': 'subject port entry',
                'dio_ECU_Din16': 'recipient port entry'},   
            2: {'dio_ECU_Din3': 'selfish light',
                'dio_ECU_Din4': 'coop light',
                'dio_ECU_Din22': 'selfish nose poke',
                'dio_ECU_Din26': 'coop nose poke',
                'dio_ECU_Din24': 'subject port entry',
                'dio_ECU_Din32': 'recipient port entry'}}

rec_to_box_dict = {'20250508_100203_Stage4_D1_1-2_merged.rec': 1,
                   '20250508_100203_Stage4_D1_1-3_merged.rec': 2,
                   '20250508_100203_Stage4_D1_2-1_merged.rec' : 1,
                   '20250508_100203_Stage4_D1_2-4_merged.rec' : 2,
                   '20250508_112121_Stage4_D1_6-1_merged.rec': 1,
                   '20250508_112121_Stage4_D1_6-3_merged.rec': 2,
                   '20250509_131426_Stage4_D1_4-1_merged.rec': 1,
                   '20250509_131426_Stage4_D1_4-2_merged.rec': 2,
                   '20250509_131426_Stage4_D1_4-3_merged.rec' :1,
                   '20250509_131426_Stage4_D1_4-4_merged.rec': 2}

In [14]:
def trodes_behaviors_to_timestamps(data, clockrate):
	'''
	Convert the data in .dat file format to normal behavior numpy array
	'''
	behavior_array = []
	event = []
	status = 1

	for (time, value) in data[1:]:
		if value[0] != status:
			raise ValueError('Invalid timestamp - the event has not started/stopped')
		
		event.append(time[0]/int(clockrate))
		if (status == 0):
			behavior_array.append(event)
			event = []
		status = 1 - status
	
	return np.array(behavior_array, dtype=float).tolist()

In [15]:
def print_n_o_events(behaviors_dict):
	for key, value in behaviors_dict.items():
		print('#', key, "-", len(value))

In [28]:
# To the folder containing .DIO folders
path = r"C:\Users\zhaoz\Desktop\Research\Cooperation\ecu_behaviors"

def load_trodes_recording(relative_path):
	'''
	Read the relative path to the folder that contains merged.DIO folder
	'''
	absolute_path = os.path.join(path, relative_path)
	name = relative_path.split('.')[0]	#20250508_100203_Stage4_D1_1-2_merged
	box = rec_to_box_dict[name + '.rec']	#1 or 2
	behaviors_dict = {}

	# Walk through the .dat files
	for root, dirs, files in os.walk(absolute_path):
		for file in files:
			if file.endswith('.dat'):
				din = file.split('.')[1]
				if din in box_to_ecu_dict[box]:	#if the din channel is meaningful
					data = tre.read_trodes_extracted_data_file(os.path.join(absolute_path, file))
					event = box_to_ecu_dict[box][din]
					behavior_data = data['data']
					# print(event, len(behavior_data))
					print(data)
					# print()
					behavior_array = trodes_behaviors_to_timestamps(behavior_data, data['clockrate'])
					behaviors_dict[event] = behavior_array

	return (name, behaviors_dict)


In [29]:
# Change it to the corresopnding recording
rec_name, behaviors = load_trodes_recording('20250509_131426_Stage4_D1_4-2_merged.DIO')

print(rec_name)
print(behaviors)
print_n_o_events(behaviors)

{'description': 'State change data for one digital channel. Display_order is 1-based', 'byte_order': 'little endian', 'original_file': '20250509_131426_Stage4_D1_4-2_merged.rec', 'clockrate': '20000', 'trodes_version': '2.3.4', 'compile_date': 'Nov 28 2022', 'compile_time': '15:10:45', 'qt_version': '6.2.2', 'commit_tag': 'heads/Release_2.3.4-0-gd5a58cd9-dirty', 'controller_firmware': '3.17', 'headstage_firmware': '2.4', 'controller_serialnum': '00104 00176', 'headstage_serialnum': '01601 00130', 'autosettle': '0', 'smartref': '0', 'gyro': '1', 'accelerometer': '1', 'magnetometer': '1', 'time_offset': '0', 'system_time_at_creation': '1746810930479', 'timestamp_at_creation': '3856874', 'first_timestamp': '3903994', 'direction': 'input', 'id': 'ECU_Din22', 'display_order': '21', 'fields': '<time uint32><state uint8>', 'data': array([([ 3903994], [0]), ([ 7851590], [1]), ([ 7854190], [0]),
       ([ 7977589], [1]), ([ 7979787], [0]), ([ 8103990], [1]),
       ([ 8105788], [0]), ([ 8239390